In [1]:
import pandas as pd
from datetime import datetime

import great_expectations as gx
from great_expectations.core.expectation_configuration import ExpectationConfiguration

from ydata_profiling import ProfileReport
from ydata_profiling.config import Settings
from ydata_profiling.model import BaseDescription, expectation_algorithms
from ydata_profiling.model.handler import Handler
from ydata_profiling.utils.dataframe import slugify
from ydata_profiling.expectations_report import ExpectationsReport
from great_expectations.checkpoint import SimpleCheckpoint

from typing import Any, Optional

import pandas as pd
from visions import VisionsTypeset
import re
import os

In [5]:
full_path = os.getcwd()
df = pd.read_csv(full_path.partition('notebooks')[0] + 'data/01_raw/Customers.csv')
df

,NewId,CustomerSince,CustomerStatus,DateOfBirth,EmploymentStatus,Gender,MaritalStatus,Placebrth,CustType,Nationality,...,Habliter,Province,District,LegalDocName1Id,LegalDocName1IdDescription,LegalIssDate,LegalExpDate,LegalIssAuth,AMLRiskRating,YrNetMonthlyIn
0,1,2012-05-23,Private Client - Standard,NaN,NaN,FEMALE,NaN,NaN,NaN,Portugal,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elevado,NaN
1,2,2022-11-30,Private Client - Standard,NaN,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medio,NaN
2,3,2022-11-30,Private Client - Standard,NaN,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Medio,NaN
3,4,2022-11-30,Private Client - Standard,NaN,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Elevado,NaN
4,5,2013-11-14,Private Client - Standard,2012-07-01,MB,MALE,OTHER,NACALA PORTO,NaN,Mozambique,...,NaN,NAMPULA,NACALA-PORTO,36.0,Licenca/Alvara,20121113,20501113,NAMPULA,Medio,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122207,122208,2016-11-29,Private Client - Standard,1973-01-16,EMPLOYED,MALE,DIVORCED,ISRAEL,RETAIL,Israel,...,Curso Superior,CABO DELGADO,MAHATE,2.0,DIRE,20240726ý20250520ý20250520,20250725ý20280519ý20280519,MIGRACAOýMAIRRO MAHATEýREEF INVESTIMENTOS,Baixo,346500.0
122208,122209,2021-07-23,Private Client - Standard,1978-01-10,EMPLOYED,MALE,SINGLE,ZIMBABWE,NaN,Zimbabwe,...,NaN,MAPUTO,MATOLA 'A',3.0,Passaporte,20170614,20270613,REPUBLIC OF ZIMBABWE,Medio,3000.0
122209,122210,2017-12-04,Private Client - Standard,1964-01-16,MB,MALE,DIVORCED,HLATHIKHULU,RETAIL,South Africa,...,NaN,NaN,NaN,3.0,Passaporte,20161013,20261012,NaN,Baixo,NaN
122210,122211,2017-12-04,Corporate - Small,NaN,NaN,NaN,NaN,NaN,RETAIL,Mozambique,...,NaN,CIDADE DE MAPUTO,BAIRRO DE SOMMERSHIELD,24.0,Certidao da Conservatoria de Regist,20151120ý20171206,20991120ý20991206,CONSERVATORIAýDIRECCAO DE CULTURA,Baixo,NaN


In [ ]:
loans = pd.read_csv(full_path.partition('notebooks')[0] + 'data/01_raw/Loans_20250331.csv')
loans

,Date,CustomerNewId,ContractId,SegmentDesc,CreditType,CreditAmount,Outstanding,CreditEOMStartDate,CreditEOMEndDate,NumberOfInstallmentsToPay,PaymentFrequency,Arreas,HasDefault
0,2025-03-31,15184,3990690,Retail,Public Sector Employee Loan,50000.00,45708.09,2024-02-29,2029-02-28,61.0,Monthly,41.0,0
1,2025-03-31,89438,6929481,Retail,Arranged Overdraft,36929.49,0.00,2024-02-29,2025-07-31,NaN,Single,0.0,0
2,2025-03-31,24924,7228322,Retail,Public Sector Employee Loan,70000.00,52951.21,2024-01-31,2027-01-31,37.0,Monthly,21.0,0
3,2025-03-31,4866,7084511,Retail,Public Sector Employee Loan,100000.00,91402.34,2024-03-31,2029-03-31,61.0,Monthly,NaN,0
4,2025-03-31,9216,6687828,Retail,Public Sector Employee Loan,30000.00,27421.48,2024-03-31,2029-03-31,61.0,Monthly,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181171,2025-03-31,49561,9535836,Retail,Public Sector Employee Loan,120000.00,110691.44,2024-03-31,2029-03-31,61.0,Monthly,26.0,0
1181172,2025-03-31,78706,4036596,Retail,Public Sector Employee Loan,100000.00,92158.60,2024-03-31,2029-03-31,61.0,Monthly,18.0,0
1181173,2025-03-31,33997,8811141,Retail,Public Sector Employee Loan,50000.00,46120.06,2024-03-31,2029-03-31,61.0,Monthly,14.0,0
1181174,2025-03-31,65758,12088744,Retail,Public Sector Employee Loan,150000.00,138237.97,2024-03-31,2029-03-31,61.0,Monthly,4.0,0


In [150]:
df.columns = [re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in df.columns]
df.columns = df.columns.str.replace('.', '', regex=False)

In [151]:
for c in ['customer_since', 'date_of_birth', 'birth_in_corp_date']:
    df[c] = pd.to_datetime(df[c], format = "%Y-%m-%d", errors="coerce")
    df[c] = df[c].apply(
        lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None
    )
    #df[c] = df[c].astype("object").where(df[c].notna(), None)


def safe_parse(val):
    try:
        if pd.isna(val):
            return None
        val_str = str(int(val))  # Convert float like 20250101.0 to '20250101'
        return datetime.strptime(val_str, "%Y%m%d").strftime('%Y-%m-%d')
    except ValueError:
        return None  # Or return None

for c in ['legal_iss_date', 'legal_exp_date']:
    df[c] = df[c].str.split('ý').str[-1]
    df[c] = df[c].apply(safe_parse)

#df['legal_exp_date'] = df['legal_exp_date'].apply(lambda x: x.strftime('%Y-%m-%d') if pd.notnull(x) else None)

for c in ['no_of_dependents', 'segment_id', 'industry_id', 'legal_doc_name1_id']:
    # Convert to Int64 (nullable int)
    df[c] = df[c].fillna(-1)
    df[c] = df[c].astype('int')

In [152]:
df.dtypes

new_id                              int64
customer_since                     object
customer_status                    object
date_of_birth                      object
employment_status                  object
gender                             object
marital_status                     object
placebrth                          object
cust_type                          object
nationality                        object
no_of_dependents                    int64
ocupation_desc                     object
residence_code                     object
residence_status                   object
residence_type                     object
segment_id                          int64
seg_group                          object
title                              object
town_country                       object
birth_in_corp_date                 object
cust_type1                         object
industry_id                         int64
habliter                           object
province                          

In [153]:
df.date_of_birth

0               None
1               None
2               None
3               None
4         2012-07-01
             ...    
122207    1973-01-16
122208    1978-01-10
122209    1964-01-16
122210          None
122211    2007-07-24
Name: date_of_birth, Length: 122212, dtype: object

## Split the data in one part as your reference dataset and the other as your analysis dataset

In [155]:
# df_ref = df[df.customer_since.dt.year != 2025]
df_ref = df[~df['customer_since'].str.startswith('2025')]
df_ref

,new_id,customer_since,customer_status,date_of_birth,employment_status,gender,marital_status,placebrth,cust_type,nationality,...,habliter,province,district,legal_doc_name1_id,legal_doc_name1_id_description,legal_iss_date,legal_exp_date,legal_iss_auth,a_m_l_risk_rating,yr_net_monthly_in
0,1,2012-05-23,Private Client - Standard,None,NaN,FEMALE,NaN,NaN,NaN,Portugal,...,NaN,NaN,NaN,-1,NaN,None,None,NaN,Elevado,NaN
1,2,2022-11-30,Private Client - Standard,None,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,-1,NaN,None,None,NaN,Medio,NaN
2,3,2022-11-30,Private Client - Standard,None,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,-1,NaN,None,None,NaN,Medio,NaN
3,4,2022-11-30,Private Client - Standard,None,NaN,MALE,NaN,NaN,NaN,Mozambique,...,NaN,NaN,NaN,-1,NaN,None,None,NaN,Elevado,NaN
4,5,2013-11-14,Private Client - Standard,2012-07-01,MB,MALE,OTHER,NACALA PORTO,NaN,Mozambique,...,NaN,NAMPULA,NACALA-PORTO,36,Licenca/Alvara,2012-11-13,2050-11-13,NAMPULA,Medio,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122207,122208,2016-11-29,Private Client - Standard,1973-01-16,EMPLOYED,MALE,DIVORCED,ISRAEL,RETAIL,Israel,...,Curso Superior,CABO DELGADO,MAHATE,2,DIRE,2025-05-20,2028-05-19,MIGRACAOýMAIRRO MAHATEýREEF INVESTIMENTOS,Baixo,346500.0
122208,122209,2021-07-23,Private Client - Standard,1978-01-10,EMPLOYED,MALE,SINGLE,ZIMBABWE,NaN,Zimbabwe,...,NaN,MAPUTO,MATOLA 'A',3,Passaporte,2017-06-14,2027-06-13,REPUBLIC OF ZIMBABWE,Medio,3000.0
122209,122210,2017-12-04,Private Client - Standard,1964-01-16,MB,MALE,DIVORCED,HLATHIKHULU,RETAIL,South Africa,...,NaN,NaN,NaN,3,Passaporte,2016-10-13,2026-10-12,NaN,Baixo,NaN
122210,122211,2017-12-04,Corporate - Small,None,NaN,NaN,NaN,NaN,RETAIL,Mozambique,...,NaN,CIDADE DE MAPUTO,BAIRRO DE SOMMERSHIELD,24,Certidao da Conservatoria de Regist,2017-12-06,2099-12-06,CONSERVATORIAýDIRECCAO DE CULTURA,Baixo,NaN


In [156]:
# df_ana = df[df.customer_since.dt.year == 2025]
df_ana = df[df['customer_since'].str.startswith('2025')]
df_ana

,new_id,customer_since,customer_status,date_of_birth,employment_status,gender,marital_status,placebrth,cust_type,nationality,...,habliter,province,district,legal_doc_name1_id,legal_doc_name1_id_description,legal_iss_date,legal_exp_date,legal_iss_auth,a_m_l_risk_rating,yr_net_monthly_in
147,148,2025-02-22,Private Client - Standard,None,NaN,NaN,NaN,NaN,NaN,Mozambique,...,NaN,SOFALA,CHAMITE,24,Certidao da Conservatoria de Regist,2025-02-18,2049-02-18,CONSERVATORIA DAS ENTIDADES LEGAISýBAU BEIRA,Baixo,NaN
150,151,2025-02-27,Corporate - Medium,None,NaN,NaN,NaN,NaN,PROSPECT,Mozambique,...,NaN,CIDADE DE MAPUTO,MALANGA,29,Certidao de Registo / Ministerio da,2025-02-13,2050-02-13,CONSERVATORIA ENTIDADES LEGAISýBAU,Elevado,NaN
175,176,2025-05-08,Private Client - Standard,None,NaN,NaN,NaN,NaN,CORPORATE,Mozambique,...,NaN,CABO DELGADO,PEMBA CIDADE,28,Licenca / Alvara para o exercicio d,2025-04-25,2049-04-25,BAUýCONSERVATORIA DAS ENTIDADES LEGAIS,Elevado,NaN
181,182,2025-02-25,Corporate - Small,None,NaN,NaN,NaN,NaN,RETAIL,Mozambique,...,NaN,CIDADE DE MAPUTO,MALHANGALAENE 'A',24,Certidao da Conservatoria de Regist,2025-02-12,2049-02-12,MINISTERIO DA JUSTICAýBAU,Medio,NaN
205,206,2025-03-29,Private Client - Standard,None,NaN,NaN,NaN,NaN,RETAIL,Mozambique,...,NaN,CIDADE DE MAPUTO,CHAMANCULO 'A',24,Certidao da Conservatoria de Regist,2023-07-13,2099-12-31,MINISTERIO DA JUSTICAýBALCAO ATENDIMENTO UNICO,Elevado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122145,122146,2025-05-16,Private Client - Standard,1981-11-11,NaN,FEMALE,OTHER,PORTUGAL,NaN,Mozambique,...,NaN,NaN,NaN,3,Passaporte,2020-10-20,2025-10-20,NaN,Elevado,NaN
122166,122167,2025-04-29,Private Client - Standard,1986-02-24,NaN,MALE,SINGLE,BEIRA,RETAIL,Mozambique,...,Bacharelato,SOFALA,BEIRA CIDADE,1,BI,2023-08-28,2028-08-27,DIC BEIRA,Elevado,NaN
122191,122192,2025-05-27,Private Client - Standard,1980-07-07,NaN,FEMALE,SINGLE,INHASSUNGE,RETAIL,Mozambique,...,Bacharelato,ZAMBEZIA,MOPEIA,1,BI,2024-07-05,2034-07-04,DIC QUELIMANE,Elevado,NaN
122197,122198,2025-03-21,Private Client - Standard,2024-08-10,NaN,FEMALE,SINGLE,MAPUTO,NaN,Mozambique,...,Ate12 ano,CIDADE DE MAPUTO,ALTO MAE 'B',1,BI,2025-02-28,2030-02-27,DNIC - MAPUTO,Baixo,NaN


In [6]:
full_path = os.getcwd()
context = gx.get_context(context_root_dir = full_path.partition('notebooks')[0] + 'gx')

In [158]:
datasource_name = "project_data_cleaned"
try:
    datasource = context.sources.add_pandas(datasource_name)
except:
    print("Data Source already exists.")
    datasource = context.datasources[datasource_name]

In [159]:
data_asset_name = "project_ana"
try:
    data_asset = datasource.add_dataframe_asset(name=data_asset_name, dataframe= df_ana)
except:
    print("The data asset already exists. The required one will be loaded.")
    data_asset = datasource.get_asset(data_asset_name)

In [160]:
data_asset_name = "project_ref"
try:
    data_asset = datasource.add_dataframe_asset(name=data_asset_name, dataframe= df_ref)
except:
    print("The data asset already exists. The required one will be loaded.")
    data_asset = datasource.get_asset(data_asset_name)

## Build expectations

In [164]:
# import logging
# from typing import Any, Dict, Tuple

# import numpy as np
# import pandas as pd
# from datetime import datetime
# import re

from great_expectations.core import ExpectationSuite, ExpectationConfiguration

# from pathlib import Path

# from kedro.config import OmegaConfigLoader
# from kedro.framework.project import settings

# conf_path = str(Path('') / settings.CONF_SOURCE)
# conf_loader = OmegaConfigLoader(conf_source=conf_path)
# credentials = conf_loader["credentials"]


# logger = logging.getLogger(__name__)

def build_expectation_suite(expectation_suite_name: str, feature_group: str) -> ExpectationSuite:
    """
    Builder used to retrieve an instance of the validation expectation suite.
    
    Args:
        expectation_suite_name (str): A dictionary with the feature group name and the respective version.
        feature_group (str): Feature group used to construct the expectations.
             
    Returns:
        ExpectationSuite: A dictionary containing all the expectations for this particular feature group.
    """
    
    expectation_suite_bank = ExpectationSuite(
        expectation_suite_name=expectation_suite_name
    )
    

    # numerical features
    if feature_group == 'numerical_features':
        for i in ['no_of_dependents', 'segment_id', 'industry_id', 'legal_doc_name1_id', 'new_id']:
                expectation_suite_bank.add_expectation(
                    ExpectationConfiguration(
                        expectation_type="expect_column_values_to_be_of_type",
                        kwargs={"column": i, "type_": "int64"},
                    )
                )
        # NewId
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_not_be_null",
                kwargs={"column": "new_id"}
            )
        )
        # YrNetMonthlyIn
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_be_of_type",
                kwargs={"column": "yr_net_monthly_in", "type_": "float64"}
            )
        )
        for i in ['yr_net_monthly_in', 'no_of_dependents', 'segment_id', 'industry_id', 'legal_doc_name1_id', 'new_id']:
            expectation_suite_bank.add_expectation(
                ExpectationConfiguration(
                    expectation_type="expect_column_values_to_be_between",
                    kwargs={
                        "column": i,
                        "min_value": -1,
                        "strict_min": False,
                        "max_value": None  # No upper bound
                    }
                )
            )
    # datetime features
    if feature_group == 'datetime_features':
        for i in ['customer_since', 'date_of_birth', 'birth_in_corp_date', 'legal_iss_date', 'legal_exp_date']:
            expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_match_strftime_format",
                kwargs={"column": i, 'strftime_format': '%Y-%m-%d'}
            )
        )
            # expectation_suite_bank.add_expectation(
            #     ExpectationConfiguration(
            #         expectation_type="expect_column_values_to_be_of_type",
            #         kwargs={"column": i, 'type_':"datetime64[ns]"},
            #     )
            # ) 
        # CustomerSince
        expectation_suite_bank.add_expectation(
            ExpectationConfiguration(
                expectation_type="expect_column_values_to_not_be_null",
                kwargs={"column": "customer_since"}
            )
        )
        # # legal_exp_date
        # expectation_suite_bank.add_expectation(
        #     ExpectationConfiguration(
        #         expectation_type="expect_column_values_to_match_strftime_format",
        #         kwargs={"column": 'legal_exp_date', 'strftime_format': '%Y-%m-%d'}
        #     )
        # )
        
    return expectation_suite_bank

In [165]:
validation_expectation_suite_numerical = build_expectation_suite("numerical_expectations", "numerical_features")
validation_expectation_suite_datetime = build_expectation_suite("datetime_expectations", "datetime_features")

In [166]:
context.add_or_update_expectation_suite(expectation_suite=validation_expectation_suite_numerical)
context.add_or_update_expectation_suite(expectation_suite=validation_expectation_suite_datetime)

{
  "expectation_suite_name": "datetime_expectations",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_values_to_match_strftime_format",
      "kwargs": {
        "column": "customer_since",
        "strftime_format": "%Y-%m-%d"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_match_strftime_format",
      "kwargs": {
        "column": "date_of_birth",
        "strftime_format": "%Y-%m-%d"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_match_strftime_format",
      "kwargs": {
        "column": "birth_in_corp_date",
        "strftime_format": "%Y-%m-%d"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_match_strftime_format",
      "kwargs": {
        "column": "legal_iss_date",
        "strftime_format": "%Y-%m-%d"
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_values_to_match_strftime_for

## Analysis

In [167]:
def get_validation_results(checkpoint_result):
    # validation_result is a dictionary containing one key-value pair
    validation_result_key, validation_result_data = next(iter(checkpoint_result["run_results"].items()))

    # Accessing the 'actions_results' from the validation_result_data
    validation_result_ = validation_result_data.get('validation_result', {})

    # Accessing the 'results' from the validation_result_data
    results = validation_result_["results"]
    meta = validation_result_["meta"]
    use_case = meta.get('expectation_suite_name')
    
    
    df_validation = pd.DataFrame({},columns=["Success","Expectation Type","Column","Column Pair","Max Value",\
                                       "Min Value","Element Count","Unexpected Count","Unexpected Percent","Value Set","Unexpected Value","Observed Value"])
    
    
    for result in results:
        # Process each result dictionary as needed
        success = result.get('success', '')
        expectation_type = result.get('expectation_config', {}).get('expectation_type', '')
        column = result.get('expectation_config', {}).get('kwargs', {}).get('column', '')
        column_A = result.get('expectation_config', {}).get('kwargs', {}).get('column_A', '')
        column_B = result.get('expectation_config', {}).get('kwargs', {}).get('column_B', '')
        value_set = result.get('expectation_config', {}).get('kwargs', {}).get('value_set', '')
        max_value = result.get('expectation_config', {}).get('kwargs', {}).get('max_value', '')
        min_value = result.get('expectation_config', {}).get('kwargs', {}).get('min_value', '')

        element_count = result.get('result', {}).get('element_count', '')
        unexpected_count = result.get('result', {}).get('unexpected_count', '')
        unexpected_percent = result.get('result', {}).get('unexpected_percent', '')
        observed_value = result.get('result', {}).get('observed_value', '')
        if type(observed_value) is list:
            #sometimes observed_vaue is not iterable
            unexpected_value = [item for item in observed_value if item not in value_set]
        else:
            unexpected_value=[]
        
        df_validation = pd.concat([df_validation, pd.DataFrame.from_dict( [{"Success" :success,"Expectation Type" :expectation_type,"Column" : column,"Column Pair" : (column_A,column_B),"Max Value" :max_value,\
                                           "Min Value" :min_value,"Element Count" :element_count,"Unexpected Count" :unexpected_count,"Unexpected Percent":unexpected_percent,\
                                                  "Value Set" : value_set,"Unexpected Value" :unexpected_value ,"Observed Value" :observed_value}])], ignore_index=True)
        
    return df_validation

In [168]:
data_asset_analysis = datasource.get_asset("project_ana")

In [169]:
batch_request = data_asset_analysis.build_batch_request()

In [170]:
checkpoint = gx.checkpoint.SimpleCheckpoint(
    name="checkpoint_datetime",
    data_context=context,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": "datetime_expectations",
        },
    ],
)
checkpoint_result = checkpoint.run()

Calculating Metrics:   0%|          | 0/41 [00:00<?, ?it/s]

In [171]:
df_validation = get_validation_results(checkpoint_result)
df_validation

,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
0,True,expect_column_values_to_match_strftime_format,customer_since,"(, )",,,9761,0,0.0,,[],
1,True,expect_column_values_to_not_be_null,customer_since,"(, )",,,9761,0,0.0,,[],
2,True,expect_column_values_to_match_strftime_format,date_of_birth,"(, )",,,9761,0,0.0,,[],
3,True,expect_column_values_to_match_strftime_format,birth_in_corp_date,"(, )",,,9761,0,0.0,,[],
4,True,expect_column_values_to_match_strftime_format,legal_iss_date,"(, )",,,9761,0,0.0,,[],
5,True,expect_column_values_to_match_strftime_format,legal_exp_date,"(, )",,,9761,0,0.0,,[],


In [ ]:
df_validation[df_validation.Success == False]

In [133]:
checkpoint_num = gx.checkpoint.SimpleCheckpoint(
    name="checkpoint_numerical",
    data_context=context,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": "numerical_expectations",
        },
    ],
)
checkpoint_result = checkpoint_num.run()

Calculating Metrics:   0%|          | 0/48 [00:00<?, ?it/s]

In [134]:
df_validation = get_validation_results(checkpoint_result)
df_validation

,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
0,True,expect_column_values_to_be_of_type,no_of_dependents,"(, )",,,,,,,[],int64
1,True,expect_column_values_to_be_between,no_of_dependents,"(, )",None,-1,9761,0,0.0,,[],
2,True,expect_column_values_to_be_of_type,segment_id,"(, )",,,,,,,[],int64
3,True,expect_column_values_to_be_between,segment_id,"(, )",None,-1,9761,0,0.0,,[],
4,True,expect_column_values_to_be_of_type,industry_id,"(, )",,,,,,,[],int64
5,True,expect_column_values_to_be_between,industry_id,"(, )",None,-1,9761,0,0.0,,[],
6,True,expect_column_values_to_be_of_type,legal_doc_name1_id,"(, )",,,,,,,[],int64
7,True,expect_column_values_to_be_between,legal_doc_name1_id,"(, )",None,-1,9761,0,0.0,,[],
8,True,expect_column_values_to_be_of_type,new_id,"(, )",,,,,,,[],int64
9,True,expect_column_values_to_not_be_null,new_id,"(, )",,,9761,0,0.0,,[],


In [72]:
df_validation[df_validation.Success == False]


,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
